In [1]:
import json
with open('./input/know.json', encoding='utf-8') as f:
    know = json.load(f)
print(know);

{'intents': {'transferir': ['Transferir dinheiro'], 'saldo': ['Qual é meu saldo?', 'Quanto tenho na conta?', 'Saldo da conta corrente', 'Quantas pilas eu tenho?', 'Qual é o saldo da minha conta?'], 'pagar-receber': ['link pra receber'], 'despedida': ['tchau', 'Até mais!'], 'recarregar-celular': ['Quero recarregar meu celular!'], 'saudacao': ['Oi', 'Olá'], 'extrato': ['Extrato da conta', 'Extrato da conta corrente']}}


In [2]:
import json
with open('./input/dialog.json', encoding='utf-8') as f:
    dialog_nodes = json.load(f)

dialog_welcome = dialog_nodes[0]

dialog_anythinelse = dialog_nodes[len(dialog_nodes) - 1]

print("dialog_welcome", dialog_welcome);
print("dialog_anythinelse", dialog_anythinelse);
print("dialog_nodes", dialog_nodes);

dialog_welcome {'condition': '#saudacao', 'output': {'text': 'Olá, eu sou o David!'}}
dialog_anythinelse {'condition': 'true', 'output': {'text': 'Desculpe, não entendi! Pode falar de outra forma?'}}
dialog_nodes [{'condition': '#saudacao', 'output': {'text': 'Olá, eu sou o David!'}}, {'condition': '#despedida', 'output': {'text': 'Muito obrigado pela atenção! Precisando estou por aqui!'}}, {'condition': '#saldo', 'output': {'text': 'Seu saldo é R$ #.###,##!'}}, {'condition': '#extrato', 'output': {'text': 'Segue seu extrato!!!'}}, {'condition': '#recarregar-celular', 'output': {'text': 'Celular recarregado!'}}, {'condition': '#pagar-receber', 'output': {'text': 'Segue o link!'}}, {'condition': '#transferir', 'output': {'text': 'Dinheiro transferido!'}}, {'condition': 'true', 'output': {'text': 'Desculpe, não entendi! Pode falar de outra forma?'}}]


In [3]:
stopwords = set(line.strip() for line in open('stopwords.txt', 'r', encoding='utf-8'))

In [4]:
import re

NON_CONTENT = r"[^\w\d\s]"
def tokenize(sentence):
    #remove non content
    sentence = re.sub(NON_CONTENT, "", sentence)
    #lower
    sentence = sentence.lower();
    
    #split
    tokens = sentence.split(" ");
        
    for sw in stopwords:
        try:
            tokens.remove(sw); 
        except ValueError:
            pass
    
    return tokens;

print(tokenize("Qual é o meu saldo?"))

['saldo']


In [12]:
intent_model = {}

for intent, samples in know["intents"].items():
    intent_model[intent] = {}
    for sample in samples:
        intent_model[intent][sample] = {
            "total" : 0,
            "tokens" : {}
        }
        for t in tokenize(sample):
            intent_model[intent][sample]["total"] += 1;
            if t in intent_model[intent][sample]["tokens"]:
                intent_model[intent][sample]["tokens"][t] += 1;
            else:
                intent_model[intent][sample]["tokens"][t] = 1;

import json
with open('./output/intent_model.json', 'w', encoding='utf-8') as outfile:
    json.dump(intent_model, outfile)
    
print("intent_model:", intent_model);    

intent_model: {'transferir': {'Transferir dinheiro': {'tokens': {'transferir': 1, 'dinheiro': 1}, 'total': 2}}, 'saldo': {'Qual é o saldo da minha conta?': {'tokens': {'saldo': 1, 'conta': 1, 'minha': 1}, 'total': 3}, 'Qual é meu saldo?': {'tokens': {'saldo': 1}, 'total': 1}, 'Quantas pilas eu tenho?': {'tokens': {'tenho': 1, 'pilas': 1}, 'total': 2}, 'Saldo da conta corrente': {'tokens': {'saldo': 1, 'corrente': 1, 'conta': 1}, 'total': 3}, 'Quanto tenho na conta?': {'tokens': {'tenho': 1, 'conta': 1}, 'total': 2}}, 'pagar-receber': {'link pra receber': {'tokens': {'link': 1, 'pra': 1, 'receber': 1}, 'total': 3}}, 'despedida': {'Até mais!': {'tokens': {'mais': 1, 'até': 1}, 'total': 2}, 'tchau': {'tokens': {'tchau': 1}, 'total': 1}}, 'recarregar-celular': {'Quero recarregar meu celular!': {'tokens': {'quero': 1, 'recarregar': 1, 'celular': 1}, 'total': 3}}, 'saudacao': {'Olá': {'tokens': {'olá': 1}, 'total': 1}, 'Oi': {'tokens': {'oi': 1}, 'total': 1}}, 'extrato': {'Extrato da conta c

In [6]:
def classify(input):
    tokens = tokenize(input);
    #print ("tokens", tokens);
    intents = {}
    for intent, samples in intent_model.items():
        intents[intent] = 0
        for s, smeta in samples.items():
            brutal_score = 0
            stokens = smeta["tokens"]
            for t in tokens:
                if t in stokens:
                    brutal_score += stokens[t]
            score = brutal_score / smeta["total"]
            if intents[intent] < score:
                intents[intent] = score
            
    intents = [{
        "intent": intent,
        "confidence": intents[intent]
    } for intent in sorted(intents, key=intents.__getitem__, reverse = True)]
    
    intents = list(filter(lambda i: i["confidence"] > 0, intents))
    
    return intents[:10];

print(classify("Qual meu saldo?"))

[{'confidence': 1.0, 'intent': 'saldo'}]


In [7]:
def nlp(input):
    return [];

In [8]:
def evalCondition(condition, context, intent, entities):
    return condition == '#' + intent;

In [9]:
def dialog(input, context, intents, entities):
    if input == "":
        return dialog_welcome;
    
    if len(intents) > 0:
        intent = intents[0]["intent"];
        for dialog_node in dialog_nodes:
            if evalCondition(dialog_node["condition"], context, intent, entities):
                return dialog_node;
    
    return dialog_anythinelse;

In [10]:
def respond(input, context = {}):
    intents = classify(input)
    entities = nlp(input)
    #print("intents", intents)
    dialog_node = dialog(input, context, intents, entities)
    return {
        "context": context,
        "intents": intents,
        "entities": entities,
        "output": dialog_node["output"]
    }

In [11]:
def test_respond(input):
    resp = respond(input);
    #print("\n\nresp", resp);
    print("\ninput: ", input, " >> ", resp["output"]["text"]);

test_respond("");
test_respond("Oi");
test_respond("Qual meu saldo?");
test_respond("Quanto tenho na conta?");
test_respond("Extrato da conta");
test_respond("link pra receber");
test_respond("Horário das agências");
test_respond("recarregar celular");
test_respond("Então tchau");


input:    >>  Olá, eu sou o David!

input:  Oi  >>  Olá, eu sou o David!

input:  Qual meu saldo?  >>  Seu saldo é R$ #.###,##!

input:  Quanto tenho na conta?  >>  Seu saldo é R$ #.###,##!

input:  Extrato da conta  >>  Segue seu extrato!!!

input:  link pra receber  >>  Segue o link!

input:  Horário das agências  >>  Desculpe, não entendi! Pode falar de outra forma?

input:  recarregar celular  >>  Celular recarregado!

input:  Então tchau  >>  Muito obrigado pela atenção! Precisando estou por aqui!
